<a href="https://colab.research.google.com/github/PondKann/CXR-Project/blob/main/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### import

In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from imutils import paths
import argparse

#### Load data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_dir = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/train'
validation_dir = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/test'

In [5]:
train_normal_dir = os.path.join(train_dir,'NORMAL')
train_pneumonia_dir = os.path.join(train_dir,'PNEUMONIA')
validation_normal_dir = os.path.join(validation_dir,'NORMAL')
validation_pneumonia_dir = os.path.join(validation_dir,'PNEUMONIA')

train_pneumonia_dir

'/content/drive/My Drive/Senior Project/xray_dataset_covid19/train/PNEUMONIA'

#### สร้างตารางเก็บชื่อภาพ และชื่อโรค 
ข้อมูลในไฟล์ train ทั้ง normal, pneunomia และ lung canser มารวมอยู่ในไฟล์ all_train แล้วลองนำมาทำ data augment

In [6]:
#ที่เก็บรูปภาพแยกเป็น 3 คลาส สำหรับ สร้างตาราง ชื่อภาพ+ชื่อคลาส 

train_normal_dir = os.path.join(train_dir,'NORMAL')
train_pneumonia_dir = os.path.join(train_dir,'PNEUMONIA')
train_ca_dir = os.path.join(train_dir,'CA')

In [7]:
all_train = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/all_train'    #ที่เก็บรูปภาพทั้งหมด ตอนนี้มี 3 คลาส

สร้าง data dict ของรูป

In [8]:
normal = []
for i in range(len(os.listdir(train_normal_dir))):
    img = os.listdir(train_normal_dir)[i]
    normal.append(img)
tnormal = pd.DataFrame({'ImageName':[normal][0],
                                       'Type1': "Normal" ,
                                       'Type2': 0                      })

In [9]:
pneumonia = []
for i in range(len(os.listdir(train_pneumonia_dir))):
    img = os.listdir(train_pneumonia_dir)[i]
    pneumonia.append(img)
tpneumonia = pd.DataFrame({'ImageName':[pneumonia][0],
                                       'Type1': "Pneumonia" ,
                                       'Type2': 1                        })

In [109]:
tlungcancer = []
for i in range(len(os.listdir(train_ca_dir))):
    img = os.listdir(train_ca_dir)[i]
    tlungcancer.append(img)
tlungcancer = pd.DataFrame({'ImageName':[tlungcancer][0],
                                       'Type1': "Lungcancer" ,
                                       'Type2': 2                        })

In [11]:
data_train2 = pd.concat([tnormal, tpneumonia])
data_train2.head(3)

,ImageName,Type1,Type2
0,IM-0006-0001.jpeg,Normal,0
1,IM-0003-0001.jpeg,Normal,0
2,IM-0001-0001.jpeg,Normal,0


In [12]:
data_train3 = pd.concat([tnormal, tpneumonia, tlungcanser])
data_train3.head(3)

,ImageName,Type1,Type2
0,IM-0006-0001.jpeg,Normal,0
1,IM-0003-0001.jpeg,Normal,0
2,IM-0001-0001.jpeg,Normal,0


# 2 class
*   #1 output 1 มิติ  ความน่าจะเป็น นำมาแปลง 0,1
*   #2 output 2 มิติ  ความน่าจะเป็นที่จะเป็นคลาส 0 หรือ 1 



In [167]:
train_img = []
for ImageName in tqdm(data_train2['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test2
    image_path = all_train +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))
    train_img.append(img)

100%|██████████| 148/148 [00:06<00:00, 22.31it/s]


### 1) output 1 มิติ  ค่า predict คือ ความน่าจะเป็น 1 ค่า 
ถ้า > 0.5 แปลงคลาส 1

ถ้า < 0.5 แปลงคลาส 0

####แบ่งข้อมูล2_1

In [168]:
data2_1 = np.array(train_img, dtype="float32") / 255.0

In [169]:
labels2_1 = np.array(data_train2[['Type1']] )  

le = LabelEncoder()                                                                      # จะแปลงชื่อ class เป็นตัวเลข
labels2_1 = le.fit_transform(labels2_1)                                                  # dimention ของ Y จะเป็น 1 คือ (37,) ต้องเปลี่ยนเป็น (37,1) ก่อน ถึงจะรัน report ได้

labels2_1.shape = (len(labels2_1), 1)                                                   #ลองดูว่าถ้าไม่แปลงเป็น 2 มิติ จะรันได้มั้ย สรุป ต้องเปลี่ยนเป็น 2 มิติอยู่ดี เพราะ predict ที่รันออกมาเป็น 2 มิติ

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [170]:
le.classes_                 #คือ ['Normal', 'Pneumonia']

array(['Normal', 'Pneumonia'], dtype=object)

In [171]:
(trainX2_1, testX2_1, trainY2_1, testY2_1) = train_test_split(data2_1, labels2_1,
										   					test_size=0.25, random_state=30)

#### train2_1

In [78]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [79]:
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.25,
		width_shift_range=0.25,
		height_shift_range=0.25,
		shear_range=0.25,
		horizontal_flip=True,
		fill_mode="nearest")

DenseNet201

In [172]:
base_model2_1 = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [173]:
base_model2_1.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model2_1.output)
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(average_pooling_layer)   #  unit = 1 คือ มิติของ output เป็น 1 มีค่าความน่าจะเป็นค่าเดียว
model2_1 = tf.keras.models.Model(inputs=base_model2_1.input, outputs=prediction_layer)

In [174]:
model2_1.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [175]:
# train the network

H2_1 = model2_1.fit(
	x=aug.flow(trainX2_1, trainY2_1, batch_size=BS),
	validation_data=(testX2_1, testY2_1),
	steps_per_epoch=len(trainX2_1) // BS,
	epochs=2)

Epoch 1/2
13/13 [==============================] - 49s 3s/step - loss: 0.6327 - accuracy: 0.6699 - val_loss: 0.6024 - val_accuracy: 0.7297
Epoch 2/2
13/13 [==============================] - 38s 3s/step - loss: 0.5866 - accuracy: 0.7282 - val_loss: 0.5825 - val_accuracy: 0.8378


#### test2_1

In [176]:
predictions2_1 = model.predict(testX2_1, batch_size=BS)

In [178]:
predictions2_1[:5]

array([[0.65554464],
       [0.42793915],
       [0.548228  ],
       [0.42028695],
       [0.5995186 ]], dtype=float32)

In [179]:
print(f' มิติของ predictions =  {predictions2_1.ndim} และ มิติของ testY =  {testY2_1.ndim}')

 มิติของ predictions =  2 และ มิติของ testY =  2


In [182]:
predictions2_1ver2 = []
for i in tqdm(predictions2_1):
    if i > 0.5:
        predictions2_1ver2.append(1)
    else:
        predictions2_1ver2.append(0)
predictions2_1ver2 = np.array(predictions2_1ver2, dtype="float32")

predictions2_1ver2.shape = (len(predictions2_1ver2) ,1)                         #แปลงให้ predictions2 เป็น 2 มิติ 
predictions2_1ver2.ndim

100%|██████████| 37/37 [00:00<00:00, 46104.95it/s]


2

In [184]:
predictions2_1ver2[:5]

array([[1.],
       [0.],
       [1.],
       [0.],
       [1.]], dtype=float32)

In [185]:
predictions2_1ver2.max(axis=1)

array([1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1.,
       1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0.,
       1., 1., 0.], dtype=float32)

In [186]:
testY2_1.max(axis=1)

array([0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1])

In [187]:
print(classification_report(testY2_1.max(axis=1),	
                            predictions2_1ver2.max(axis=1),  
							target_names=le.classes_ ))

              precision    recall  f1-score   support

      Normal       0.15      0.11      0.13        18
   Pneumonia       0.33      0.42      0.37        19

    accuracy                           0.27        37
   macro avg       0.24      0.27      0.25        37
weighted avg       0.25      0.27      0.25        37



## 2) output 2 มิติ : ค่า predict คือ ความน่าจะเป็น 2 ค่า (ความน่าจะเป็นที่จะเป็นคลาส 0 และ 1)

In [188]:
data2_2 = np.array(train_img, dtype="float32") / 255.0

In [189]:
labels2_2 = np.array(data_train2[['Type1']] )         

In [190]:
le2_2 = LabelEncoder()                                                                   #จะแปลงชื่อ class เป็นตัวเลข
labels2_2 = le2_2.fit_transform(labels2_2)                                                  # dimention ของ Y จะเป็น 1 (37,) 
labels2_2 = to_categorical(labels2_2, num_classes=2)                            # แปลงเป็นเลข ดัมมี่ (1เป็น 1,0 ----- 0 เป็น  0,1)    
            #shape (148, 2) ค่าเดิม 0. กับ 1.  หลังจากรันจะเป็น [0., 1.] กับ [1., 0.]  ------->  แต่จะรัน model2.fit ไม่ได้ อาจเพราะเป็นโมเดลสำหรับ output แค่ 1 ค่า 

(trainX2_2, testX2_2, trainY2_2, testY2_2) = train_test_split(data2_2, labels2_2,
	                                                            test_size=0.25, random_state=40)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [191]:
le2_2.classes_

array(['Normal', 'Pneumonia'], dtype=object)

### train2_2

In [192]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [193]:
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.25,
		width_shift_range=0.25,
		height_shift_range=0.25,
		shear_range=0.25,
		horizontal_flip=True,
		fill_mode="nearest")

DenseNet201

In [194]:
base_model2_2 = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [197]:
base_model2_2.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model2_2.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # ลองแก้ unit จาก 1 เป็น 2
model2_2 = tf.keras.models.Model(inputs=base_model2_2.input, outputs=prediction_layer)

In [198]:
model2_2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [199]:
H2_2 = model2_2.fit(
	x=aug.flow(trainX2_2, trainY2_2, batch_size=BS),
	validation_data=(testX2_2, testY2_2),
	steps_per_epoch=len(trainX2_2) // BS,
	epochs=2)

Epoch 1/2
13/13 [==============================] - 50s 3s/step - loss: 0.7842 - accuracy: 0.5340 - val_loss: 0.7843 - val_accuracy: 0.3243
Epoch 2/2
13/13 [==============================] - 37s 3s/step - loss: 0.7629 - accuracy: 0.4854 - val_loss: 0.7519 - val_accuracy: 0.3784


### test

In [200]:
predictions2_2 = model2_2.predict(testX2_2, batch_size=BS)

In [201]:
predictions2_2.ndim

2

In [202]:
predictions2_2[:5]

array([[0.4065986 , 0.5248565 ],
       [0.39844054, 0.43749118],
       [0.4478856 , 0.3925465 ],
       [0.42513436, 0.4596733 ],
       [0.42985493, 0.4560034 ]], dtype=float32)

In [204]:
print(classification_report(testY2_2.argmax(axis=1),	
                            predictions2_2.argmax(axis=1),  
							target_names=le2_2.classes_ ))

              precision    recall  f1-score   support

      Normal       0.40      0.56      0.47        18
   Pneumonia       0.33      0.21      0.26        19

    accuracy                           0.38        37
   macro avg       0.37      0.38      0.36        37
weighted avg       0.37      0.38      0.36        37



## Fine tune

In [ ]:
len(base_model2.layers)

707

In [ ]:
base_model2.trainable = True
for layer in base_model2.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

In [ ]:
model2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
#train
model2.fit_generator(train_generator, epochs=10, 
                    validation_data=valid_generator,
                    callbacks=[callback2])

# 3 class ==>  DenseNet201

###แบ่งข้อมูล3

In [205]:
data_train3 = pd.concat([tnormal, tpneumonia, tlungcancer])
data_train3.head(3)

,ImageName,Type1,Type2
0,IM-0006-0001.jpeg,Normal,0
1,IM-0003-0001.jpeg,Normal,0
2,IM-0001-0001.jpeg,Normal,0


In [206]:
train_img3 = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = all_train +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))
    train_img3.append(img)


100%|██████████| 180/180 [00:07<00:00, 24.07it/s]


In [207]:
data3 = np.array(train_img3, dtype="float32") / 255.0

In [208]:
labels3 = np.array(data_train3[['Type1']] )           #shape (180, 1) ค่าเป็น ชื่อคลาส Normal, Pneumonia , Lungcancer

In [209]:
le3 = LabelEncoder()                                                                     
labels3 = le3.fit_transform(labels3)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 
labels3 = to_categorical(labels3, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [210]:
(trainX3, testX3, trainY3, testY3) = train_test_split(data3, labels3,
	                                        test_size=0.25, random_state=52)

In [211]:
le3.classes_

array(['Lungcancer', 'Normal', 'Pneumonia'], dtype=object)

## train3

In [212]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [213]:
# initialize an our data augmenter as an "empty" image data generator
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.15,
		height_shift_range=0.15,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

DenseNet201

In [214]:
base_model3 = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [133]:
base_model3.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model3 = tf.keras.models.Model(inputs=base_model3.input, outputs=prediction_layer)

In [215]:
model3.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [216]:
H3 = model3.fit(
	x=aug.flow(trainX3, trainY3, batch_size=BS),
	validation_data=(testX3, testY3),
	steps_per_epoch=len(trainX3) // BS,
	epochs=5)

Epoch 1/5
16/16 [==============================] - 56s 3s/step - loss: 0.4303 - accuracy: 0.8346 - val_loss: 0.4048 - val_accuracy: 0.8667
Epoch 2/5
16/16 [==============================] - 41s 3s/step - loss: 0.4119 - accuracy: 0.8504 - val_loss: 0.3896 - val_accuracy: 0.8889
Epoch 3/5
16/16 [==============================] - 42s 3s/step - loss: 0.3836 - accuracy: 0.9375 - val_loss: 0.3756 - val_accuracy: 0.8889
Epoch 4/5
16/16 [==============================] - 43s 3s/step - loss: 0.3867 - accuracy: 0.8583 - val_loss: 0.3620 - val_accuracy: 0.8889
Epoch 5/5
16/16 [==============================] - 43s 3s/step - loss: 0.3712 - accuracy: 0.8898 - val_loss: 0.3479 - val_accuracy: 0.9556


## test3

In [217]:
predictions3 = model3.predict(testX3, batch_size=BS)

In [219]:
predictions3[:10]

array([[0.2131609 , 0.7493581 , 0.3810581 ],
       [0.46792397, 0.10656792, 0.41420412],
       [0.09939459, 0.7906554 , 0.25583833],
       [0.15035456, 0.20362276, 0.6627989 ],
       [0.10807812, 0.19862849, 0.52380246],
       [0.15961602, 0.18624142, 0.46225345],
       [0.2020602 , 0.49919894, 0.5117604 ],
       [0.14736903, 0.6294489 , 0.16581935],
       [0.16934818, 0.80198014, 0.16803303],
       [0.24629155, 0.11937389, 0.5752012 ]], dtype=float32)

In [225]:
testY3.argmax(axis=1)

array([1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 0, 1, 1, 2,
       0, 0, 1, 1, 2, 1, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 1, 1, 2, 1, 1, 1,
       2])

In [228]:
print(classification_report(testY3.argmax(axis=1),	
                           predictions3.argmax(axis=1),  
                            target_names=le3.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.89      1.00      0.94         8
      Normal       0.95      1.00      0.97        18
   Pneumonia       1.00      0.89      0.94        19

    accuracy                           0.96        45
   macro avg       0.95      0.96      0.95        45
weighted avg       0.96      0.96      0.96        45



# 3 class ==> InceptionV3

###แบ่งข้อมูล3

In [ ]:
data_train3 = pd.concat([tnormal, tpneumonia, tlungcancer])
data_train3.head(3)

,ImageName,Type1,Type2
0,IM-0006-0001.jpeg,Normal,0
1,IM-0003-0001.jpeg,Normal,0
2,IM-0001-0001.jpeg,Normal,0


In [ ]:
train_img3 = []
for ImageName in tqdm(data_train3['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test3
    image_path = all_train +'/' + ImageName                          # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))
    train_img3.append(img)


100%|██████████| 180/180 [00:07<00:00, 24.07it/s]


In [ ]:
data3 = np.array(train_img3, dtype="float32") / 255.0

In [ ]:
labels3 = np.array(data_train3[['Type1']] )           #shape (180, 1) ค่าเป็น ชื่อคลาส Normal, Pneumonia , Lungcancer

In [ ]:
le3 = LabelEncoder()                                                                     
labels3 = le3.fit_transform(labels3)                                         #จะแปลงชื่อ class เป็นตัวเลข 1, 2, 0 
labels3 = to_categorical(labels3, num_classes=3)                   #shape (180, 3) มีค่าเป็น  [0., 1., 0.]  กับ   [0., 0., 1.] กับ  [1., 0., 0.]

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
(trainX3, testX3, trainY3, testY3) = train_test_split(data3, labels3,
	                                        test_size=0.25, random_state=52)

In [ ]:
le3.classes_

array(['Lungcancer', 'Normal', 'Pneumonia'], dtype=object)

## train3

In [ ]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [ ]:
# initialize an our data augmenter as an "empty" image data generator
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.15,
		height_shift_range=0.15,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

InceptionV3

In [238]:
base_model3_2 = tf.keras.applications.InceptionV3(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [239]:
base_model3_2.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model3_2.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="sigmoid")(average_pooling_layer)   # unit 3
model3_2 = tf.keras.models.Model(inputs=base_model3_2.input, outputs=prediction_layer)

In [240]:
model3_2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [241]:
H3_2 = model3_2.fit(
	x=aug.flow(trainX3, trainY3, batch_size=BS),
	validation_data=(testX3, testY3),
	steps_per_epoch=len(trainX3) // BS,
	epochs=5)

Epoch 1/5
16/16 [==============================] - 66s 1s/step - loss: 0.6456 - accuracy: 0.3622 - val_loss: 0.6885 - val_accuracy: 0.3111
Epoch 2/5
16/16 [==============================] - 20s 1s/step - loss: 0.6100 - accuracy: 0.4609 - val_loss: 0.6580 - val_accuracy: 0.4222
Epoch 3/5
16/16 [==============================] - 19s 1s/step - loss: 0.5763 - accuracy: 0.4724 - val_loss: 0.6367 - val_accuracy: 0.4444
Epoch 4/5
16/16 [==============================] - 19s 1s/step - loss: 0.5367 - accuracy: 0.6220 - val_loss: 0.6169 - val_accuracy: 0.4667
Epoch 5/5
16/16 [==============================] - 19s 1s/step - loss: 0.5387 - accuracy: 0.5433 - val_loss: 0.5953 - val_accuracy: 0.4889


## test3

In [245]:
predictions3_2 = model3_2.predict(testX3, batch_size=BS)

In [246]:
predictions3_2[:10]

array([[0.14766952, 0.17961118, 0.39483184],
       [0.1772252 , 0.5482553 , 0.2570692 ],
       [0.09218809, 0.35767338, 0.5883238 ],
       [0.29868764, 0.32585305, 0.43447497],
       [0.172254  , 0.26656708, 0.63736844],
       [0.24141678, 0.22357512, 0.41647854],
       [0.15244225, 0.17420259, 0.5500125 ],
       [0.18643695, 0.6620075 , 0.3693062 ],
       [0.09739217, 0.3352247 , 0.60741   ],
       [0.0999496 , 0.27409285, 0.5338486 ]], dtype=float32)

In [ ]:
testY3.argmax(axis=1)

array([1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 0, 1, 1, 2,
       0, 0, 1, 1, 2, 1, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 1, 1, 2, 1, 1, 1,
       2])

In [247]:
print(classification_report(testY3.argmax(axis=1),	
                           predictions3_2.argmax(axis=1),  
                            target_names=le3.classes_))

              precision    recall  f1-score   support

  Lungcancer       0.00      0.00      0.00         8
      Normal       0.58      0.39      0.47        18
   Pneumonia       0.45      0.79      0.58        19

    accuracy                           0.49        45
   macro avg       0.35      0.39      0.35        45
weighted avg       0.43      0.49      0.43        45



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
